In [7]:
from urllib import request
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits, ascii
import os
import numpy as np

import pickle
import urllib
import requests

import seaborn as sns
from astropy.time import Time
%matplotlib inline

pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
!pip3 install pandas --upgrade

Requirement already up-to-date: pandas in ./anaconda3/lib/python3.8/site-packages (1.5.0)


In [3]:
colnames=['Flare','Start', 'time',        'Peak','End',      'Dur',   'Peak c/s',     'Total',      'Energy', 'X Pos', 'Y Pos', 'Radial',   'AR',    'Flag1','Flag2','Flag3','Flag4','Flag5','Flag6','Flag7','Flag8','Flag9','Flag10','Flag11']

In [4]:

flares1=pd.read_table('https://hesperia.gsfc.nasa.gov/hessidata/dbase/hessi_flare_list.txt',names=colnames,
                      skipfooter=40,sep= '\s+',skiprows=6)
flares1

<ipython-input-4-b78d682c36c3>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
  flares1=pd.read_table('https://hesperia.gsfc.nasa.gov/hessidata/dbase/hessi_flare_list.txt',names=colnames,


,Flare,Start,time,Peak,End,Dur,Peak c/s,Total,Energy,X Pos,...,Flag2,Flag3,Flag4,Flag5,Flag6,Flag7,Flag8,Flag9,Flag10,Flag11
0,2021229,12-Feb-2002,02:15:24,02:19:22,02:25:48,624,46,75270,25-50,958,...,P1,None,None,None,None,None,None,None,None,None
1,2021230,12-Feb-2002,02:49:04,02:49:38,02:50:48,104,20,6528,6-12,637,...,EE,P1,Q1,None,None,None,None,None,None,None
2,2021213,12-Feb-2002,21:30:08,21:33:46,21:41:16,668,147,184773,12-25,597,...,P1,None,None,None,None,None,None,None,None,None
3,2021332,13-Feb-2002,00:53:28,00:54:54,00:56:52,204,16,12070,12-25,-309,...,P1,None,None,None,None,None,None,None,None,None
4,2021356,13-Feb-2002,02:48:56,02:49:34,02:50:36,100,13,4995,12-25,-385,...,P1,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104031,18021101,11-Feb-2018,01:07:04,01:08:30,01:10:08,184,312,33349,12-25,11,...,DF,DR,P1,PE,Q3,None,None,None,None,None
104032,18021203,12-Feb-2018,06:58:00,06:59:10,07:01:04,184,181,19930,6-12,290,...,DF,DR,P1,PE,Q3,None,None,None,None,None
104033,18022601,26-Feb-2018,15:49:40,15:51:14,15:53:56,256,155,16984,6-12,-15,...,DF,P1,PE,Q2,None,None,None,None,None,None
104034,18030301,3-Mar-2018,04:04:24,04:06:10,04:09:52,328,116,19161,6-12,924,...,P1,PE,Q1,None,None,None,None,None,None,None


In [8]:
# import astropy.time
from astropy.time import Time

# read in data
flares1=pd.read_table('https://hesperia.gsfc.nasa.gov/hessidata/dbase/hessi_flare_list.txt',names=colnames,
                    skipfooter=40,sep= '\s+',skiprows=6, engine='python')

# cut out data before July 28, 2015, 1 day before the first of our solar spectra were taken
# see https://dace.unige.ch/observationSearch/?observationType=[%22solarSpectroscopy%22]

# in the column 'start', find 28-Jul-2015 
list = flares1.index[flares1['Start']== '28-Jul-2015'].tolist()

# use iloc to cut out all date before that start time
flares1_ex = flares1.iloc[list[0]:]


# combine all the flag columns into 1 column, use combine method w lambda
# ref: https://stackoverflow.com/questions/33098383/merge-multiple-column-values-into-one-column-in-python-pandas

# find column index of Flag1
index = flares1_ex.columns.get_loc("Flag1")

# combine columns and add new column to flares1_ex
flares1_ex['Flags'] = flares1_ex[flares1_ex.columns[index:]].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)


# convert all times to JD with astropy.time module
# create three new columns with the start, end, and peak datetimes in JD format, do this first (?)
# ref: https://docs.astropy.org/en/stable/time/index.html

# convert date to iso format https://datascientyst.com/convert-month-number-to-month-name-pandas-dataframe/
flares1_ex['Start'] = pd.to_datetime(flares1_ex['Start'])

# create a list of times
time_list = flares1['time'].tolist()

# combine columns 
flares1_ex["time JD"] = flares1_ex["Start"].astype(str) + ' ' + flares1_ex["time"].astype(str)
list1 = flares1_ex["time JD"].tolist()
t1 = Time(list1,format = 'iso', scale = 'utc')

# convert to JD
time_JD = pd.Series(t1.jd)
flares1_ex['time JD'] = time_JD.values

# repeat the process w list of peak
flares1_ex["Peak JD"] = flares1_ex["Start"].astype(str) + ' ' + flares1_ex["Peak"].astype(str)
list2 = flares1_ex["Peak JD"].tolist()
t2 = Time(list2,format = 'iso', scale = 'utc')

Peak_JD = pd.Series(t2.jd)
flares1_ex['Peak JD'] = Peak_JD.values

# list of end
flares1_ex["End JD"] = flares1_ex["Start"].astype(str) + ' ' + flares1_ex["End"].astype(str)
list3 = flares1_ex["End JD"].tolist()
t3 = Time(list3,format = 'iso', scale = 'utc')

End_JD = pd.Series(t3.jd)
flares1_ex['End JD'] = End_JD.values

# show extracted data
flares1_ex


#export to file
flares1_ex.to_csv("flares1_extract.csv", index = False) 

In [9]:
# combine flag columns
df_ex = flares1_ex

index1 = df_ex.columns.get_loc("Flag1")
index2 = df_ex.columns.get_loc("Flag11")

df_ex['Flags'] = df_ex[df_ex.columns[index1:index2+1]].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)

# remove 11 flag columns
df_ex.drop(['Flag1','Flag2','Flag3','Flag4','Flag5','Flag6','Flag7','Flag8','Flag9','Flag10','Flag11'], axis=1, inplace=True)    

df_ex

,Flare,Start,time,Peak,End,Dur,Peak c/s,Total,Energy,X Pos,Y Pos,Radial,AR,Flags,time JD,Peak JD,End JD
96217,15072801,2015-07-28,00:37:32,00:41:42,00:45:32,480,240,279476,6-12,-331,-294,443,2389,"A0, P1, PE, Q2, SE",2.457232e+06,2.457232e+06,2.457232e+06
96218,15072808,2015-07-28,03:05:00,03:05:30,03:05:52,52,31,8004,6-12,254,-372,451,2390,"A0, DF, P1, PE, Q2",2.457232e+06,2.457232e+06,2.457232e+06
96219,15072803,2015-07-28,03:14:12,03:17:06,03:22:12,480,40,77107,6-12,-317,-275,420,2389,"A0, DF, P1, PE, Q2",2.457232e+06,2.457232e+06,2.457232e+06
96220,15072802,2015-07-28,08:00:04,08:01:34,08:05:08,304,51,61246,6-12,-264,-281,386,2389,"A0, DF, P1, PE, Q2",2.457232e+06,2.457232e+06,2.457232e+06
96221,15072804,2015-07-28,12:32:40,12:34:10,12:37:40,300,67,71406,6-12,314,-314,444,2390,"A0, DR, P1, PE, Q2",2.457232e+06,2.457232e+06,2.457232e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104031,18021101,2018-02-11,01:07:04,01:08:30,01:10:08,184,312,33349,12-25,11,-86,87,2699,"A0, DF, DR, P1, PE, Q3",2.458161e+06,2.458161e+06,2.458161e+06
104032,18021203,2018-02-12,06:58:00,06:59:10,07:01:04,184,181,19930,6-12,290,11,290,2699,"A0, DF, DR, P1, PE, Q3",2.458162e+06,2.458162e+06,2.458162e+06
104033,18022601,2018-02-26,15:49:40,15:51:14,15:53:56,256,155,16984,6-12,-15,199,199,0,"A0, DF, P1, PE, Q2",2.458176e+06,2.458176e+06,2.458176e+06
104034,18030301,2018-03-03,04:04:24,04:06:10,04:09:52,328,116,19161,6-12,924,129,933,2700,"A0, P1, PE, Q1",2.458181e+06,2.458181e+06,2.458181e+06


In [11]:
# remove rows that have zeroes for both x-pos and y-pos

df_ex_1 = df_ex[df_ex['X Pos'] != 0]
df_ex_2 = df_ex_1[df_ex_1['Y Pos'] != 0]
df_ex_2

,Flare,Start,time,Peak,End,Dur,Peak c/s,Total,Energy,X Pos,Y Pos,Radial,AR,Flags,time JD,Peak JD,End JD
96217,15072801,2015-07-28,00:37:32,00:41:42,00:45:32,480,240,279476,6-12,-331,-294,443,2389,"A0, P1, PE, Q2, SE",2.457232e+06,2.457232e+06,2.457232e+06
96218,15072808,2015-07-28,03:05:00,03:05:30,03:05:52,52,31,8004,6-12,254,-372,451,2390,"A0, DF, P1, PE, Q2",2.457232e+06,2.457232e+06,2.457232e+06
96219,15072803,2015-07-28,03:14:12,03:17:06,03:22:12,480,40,77107,6-12,-317,-275,420,2389,"A0, DF, P1, PE, Q2",2.457232e+06,2.457232e+06,2.457232e+06
96220,15072802,2015-07-28,08:00:04,08:01:34,08:05:08,304,51,61246,6-12,-264,-281,386,2389,"A0, DF, P1, PE, Q2",2.457232e+06,2.457232e+06,2.457232e+06
96221,15072804,2015-07-28,12:32:40,12:34:10,12:37:40,300,67,71406,6-12,314,-314,444,2390,"A0, DR, P1, PE, Q2",2.457232e+06,2.457232e+06,2.457232e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104031,18021101,2018-02-11,01:07:04,01:08:30,01:10:08,184,312,33349,12-25,11,-86,87,2699,"A0, DF, DR, P1, PE, Q3",2.458161e+06,2.458161e+06,2.458161e+06
104032,18021203,2018-02-12,06:58:00,06:59:10,07:01:04,184,181,19930,6-12,290,11,290,2699,"A0, DF, DR, P1, PE, Q3",2.458162e+06,2.458162e+06,2.458162e+06
104033,18022601,2018-02-26,15:49:40,15:51:14,15:53:56,256,155,16984,6-12,-15,199,199,0,"A0, DF, P1, PE, Q2",2.458176e+06,2.458176e+06,2.458176e+06
104034,18030301,2018-03-03,04:04:24,04:06:10,04:09:52,328,116,19161,6-12,924,129,933,2700,"A0, P1, PE, Q1",2.458181e+06,2.458181e+06,2.458181e+06


In [ ]:
# Remove observations that have the flags for high noise data

df_ex_2 = df_ex_1[df_ex_1['Y Pos'] != 0]
df_ex_2

In [12]:
# save as csv file

df_ex_2.head()


,Flare,Start,time,Peak,End,Dur,Peak c/s,Total,Energy,X Pos,Y Pos,Radial,AR,Flags,time JD,Peak JD,End JD
96217,15072801,2015-07-28,00:37:32,00:41:42,00:45:32,480,240,279476,6-12,-331,-294,443,2389,"A0, P1, PE, Q2, SE",2.457232e+06,2.457232e+06,2.457232e+06
96218,15072808,2015-07-28,03:05:00,03:05:30,03:05:52,52,31,8004,6-12,254,-372,451,2390,"A0, DF, P1, PE, Q2",2.457232e+06,2.457232e+06,2.457232e+06
96219,15072803,2015-07-28,03:14:12,03:17:06,03:22:12,480,40,77107,6-12,-317,-275,420,2389,"A0, DF, P1, PE, Q2",2.457232e+06,2.457232e+06,2.457232e+06
96220,15072802,2015-07-28,08:00:04,08:01:34,08:05:08,304,51,61246,6-12,-264,-281,386,2389,"A0, DF, P1, PE, Q2",2.457232e+06,2.457232e+06,2.457232e+06
96221,15072804,2015-07-28,12:32:40,12:34:10,12:37:40,300,67,71406,6-12,314,-314,444,2390,"A0, DR, P1, PE, Q2",2.457232e+06,2.457232e+06,2.457232e+06


In [14]:
# save as csv file
df_ex_2.to_csv("/home/nhao/flares/flares_data_new.csv", index = False)